Data for this notebook can be loaded here: https://www.kaggle.com/c/dogs-vs-cats/data

Extract this archive to `data/` folder or change `data_path`.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import os
import numpy as np


from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
from torchvision import transforms
from torchvision.transforms import Compose, Resize, ToTensor, Normalize, RandomResizedCrop, RandomHorizontalFlip 

from PIL import Image
from sklearn.model_selection import train_test_split

In [2]:
data_path = 'data'
train_path = os.path.join(data_path, 'train')
test_path = os.path.join(data_path, 'test')

In [ ]:
train_img_list, valid_img_list = train_test_split(os.listdir(train_path), test_size=0.2, random_state=42, shuffle=True)

In [ ]:
batch_size = 64

### Pytorch model

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
class TrainDataset(Dataset):
    def __init__(self, path, img_list, transform=None):
        self.path = path
        self.img_list = img_list
        self.transform = transform
    
    def __len__(self):
        return len(self.img_list)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.path, self.img_list[idx])
        
        image = Image.open(img_path)
        label = 0 if self.img_list[idx].startswith('cat') else 1
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [7]:
train_transforms =  transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])

valid_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

In [15]:
train_ds = TrainDataset(train_path, train_img_list, transform=train_transforms)
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

valid_ds = TrainDataset(train_path, valid_img_list, transform=valid_transforms)
valid_loader = DataLoader(valid_ds, batch_size=batch_size, shuffle=True)

In [21]:
class Cnn(nn.Module):
    def __init__(self):
        super(Cnn,self).__init__()
        
        # # [W - K + 2*P] / S + 1
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,16,kernel_size=3, padding=0,stride=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        
        self.layer2 = nn.Sequential(
            nn.Conv2d(16,32, kernel_size=3, padding=0, stride=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
            )

        self.layer3 = nn.Sequential(
            nn.Conv2d(32,64, kernel_size=3, padding=0, stride=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        
        self.fc1 = nn.Linear(3*3*64,10)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(10,2)
        self.relu = nn.ReLU()
        
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1)
        out = self.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [22]:
import torch.optim as optim

model = Cnn().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params = model.parameters(),lr=0.001)

In [23]:
epochs = 10

for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0
    
    for data, label in train_loader:
        data = data.to(device)
        label = label.to(device)
        
        output = model(data)
        loss = criterion(output, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        acc = ((output.argmax(dim=1) == label).float().mean())
        epoch_accuracy += acc/len(train_loader)
        epoch_loss += loss/len(train_loader)
        
    print('Epoch : {}, train accuracy : {}, train loss : {}'.format(epoch+1, epoch_accuracy,epoch_loss))
    
    with torch.no_grad():
        epoch_val_accuracy=0
        epoch_val_loss =0
        for data, label in valid_loader:
            data = data.to(device)
            label = label.to(device)
            
            val_output = model(data)
            val_loss = criterion(val_output,label)
            
            
            acc = ((val_output.argmax(dim=1) == label).float().mean())
            epoch_val_accuracy += acc/ len(val_loader)
            epoch_val_loss += val_loss/ len(val_loader)
            
        print('Epoch : {}, val_accuracy : {}, val_loss : {}'.format(epoch+1, epoch_val_accuracy,epoch_val_loss))

Epoch : 1, train accuracy : 0.6380994915962219, train loss : 0.6324897408485413
Epoch : 2, train accuracy : 0.6986505389213562, train loss : 0.568233072757721
Epoch : 3, train accuracy : 0.7241005301475525, train loss : 0.5408569574356079
Epoch : 4, train accuracy : 0.7350507974624634, train loss : 0.523330807685852
Epoch : 5, train accuracy : 0.7425004243850708, train loss : 0.5077390074729919
Epoch : 6, train accuracy : 0.7471500635147095, train loss : 0.5068618655204773
Epoch : 7, train accuracy : 0.7600006461143494, train loss : 0.48727166652679443
Epoch : 8, train accuracy : 0.7645007371902466, train loss : 0.4776691496372223
Epoch : 9, train accuracy : 0.7688007950782776, train loss : 0.4730830788612366
Epoch : 10, train accuracy : 0.7774502635002136, train loss : 0.4613279700279236


In [26]:
PATH = './cats.pth'
torch.save(model.state_dict(), PATH)